In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] ="6"
import json
import sys
import torch
import random
import numpy as np
import pandas as pd
import csv
import spacy
from tqdm import tqdm
import json
from collections import Counter
import time
import spacy
import re

nlp = spacy.load("en_core_web_sm")

/nas/home/yifjia/ENTER/envs/tripPy_copy/lib/python3.7/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


#### read prompting file

In [2]:
code_generation = np.load('./atomic_prompting/prompting_result_200.npy',allow_pickle=True)

In [3]:
def splitfeedback(sample):
    feedback = sample['feedback']
    step_result = {}
    for step_feedback in feedback.split('\n    def ')[1:]:
        key = step_feedback.replace('()','').split(':')[0]
        ### previous setting of temp is 0.7, which may bring some random \n\n main
        code_step =  [_.lstrip() for _ in step_feedback.split('\n\n')[0].split('\n ')[1:]]
        step_result[key] = code_step
        
    return step_result

In [4]:
for item in code_generation:
    item['step_result'] = splitfeedback(item)

#### get raw operation and parameters

In [8]:
operation_set = set()
parameter_set = set()
operation_dict = {}
parameter_dict = {}
for index,sample in enumerate(code_generation):
        sample_result = sample['step_result']
        for sen,step in sample_result.items():
            for code in step:
                try:
                    operation = code.split(' = ')[1].split('(')[0].lstrip().lower()
                    operation_set.add(operation.lstrip().rstrip())
                    if operation == 'strain':
                                print(sen+":"+code)
                    operation_dict[operation] = operation_dict[operation] + 1 if operation in operation_dict.keys() else 1
                    parameter_list = code.split('(')[1].split(')')[0].split(',')
                    for step_parameter in parameter_list:
                        if '=' in step_parameter:
                            parameter = step_parameter.split('=')[0].lstrip()
                            if parameter == 'ingredient':
                                print(index)
                            parameter_set.add(parameter)
                            parameter_dict[parameter] = parameter_dict[parameter] + 1 if parameter in parameter_dict.keys() else 1
                except:
                    print('wrong at {} because {}'.format(index,code))

wrong at 10 because Out = h10
23
23
Add_vodka_and_strain_through_a_colander:h11 = Strain(h10, container=colander)
Strain_into_a_bowl,_allow_to_cool_slightly,_then_beat_in_the_soft_butter,_a_small_piece_at_a_time:h20 = Strain(h19, container=bowl)
wrong at 103 because #Yield: 2/3 cup.
wrong at 111 because Out = h7
Fill_with_margarita_mix,_shake_well,_and_strain_into_glasses:h8 = Strain(h7, container=glasses)
Strain_the_soup_through_a_fine_sieve:h12 = Strain(h11, how=through_a_fine_sieve)


#### selected operation and parameter

In [9]:
operation_dict = sorted(operation_dict.items(),key = lambda x:x[1],reverse = True)
parameter_dict = sorted(parameter_dict.items(),key = lambda x:x[1],reverse = True)

In [11]:
sum([int(_[1]) for _ in operation_dict])

2678

In [12]:
sum([int(_[1]) for _ in operation_dict[:60]])

2248

In [13]:
2248/2678

0.8394324122479462

In [16]:
Operation_Dict = {'add':['add','top'],
                 'serve':['serve'],
                 'stir':['stir'],
                 'cook':['cook','saute'],
                 'place':['place','transfer','put','spoon','arrange','line','set','set_aside','return','let_stand','strain'],
                 'bake':['bake','grill','toast','roast','broil'],
                 'mix':['mix','combine','pour','toss','blend','puree','cream','shake'],
                 'preheat':['preheat'],
                 'heat':['heat','melt','brown','microwave'],
                 'cool':['cool','refrigerate','reduce','chill','freeze'],
                 'remove':['remove'],
                 'cover':['cover','seal'],
                 'sprinkle':['sprinkle','whisk'],
                 'cut':['cut','divide','slice','break','chop','peel','dice'],
                 'boil':['boil'],
                 'beat' :['beat'],
                 'simmer':['simmer'],
                 'drain':['drain'],
                 'season': ['season'],
                 'shape' : ['shape','fold','roll','press','knead','flatten','rise'],
                 'garnish':['garnish'],
                 'fry':['fry'],
                 'fill':['fill'],
                 'spread':['rub','spread','dip','grease','coat','brush'],
                 'spray':['spray','drizzle','scatter'],
                 'filt':['sift','scoop'],
                 'crush':['crush','mash'],
                 'steam':['steam'],
                 'dissolve':['dissolve'],
                 'make':['make','process','check','store','turn','measure']
                 }

Parameter_list = ['how','container','time','temp','tool','size','amount','speed','salt','pepper']

In [17]:
Operation_reverse_dict = {}
for key,value_list in Operation_Dict.items():
    for value in value_list:
        Operation_reverse_dict[value]=key

In [20]:
sample = code_generation[0]
step_result = sample['step_result']
step_triple = []
# (input , operation_dict, output)
for sen,step_func_list in step_result.items():
    for step_func in step_func_list:
        step_output = step_func.split(" = ")[0]
        operation_detail = {}
        step_operation = step_func.split(" = ")[1].split("(")[0].lower()
        operation_detail['orginal_step_operation']=step_operation
        step_operation = Operation_reverse_dict.get(step_operation,'make').capitalize()  
        operation_detail['step_operation']=step_operation
        function_detail = step_func.split('(')[1].split(')')[0]
        step_input = [_.strip() for _ in function_detail.split(', ') if "=" not in _]
        step_parameter = [_.strip() for _ in function_detail.split(', ') if "=" in _]
        step_parameter,_ = filterParameter(step_parameter)
        if _ is not None:
            step_input.append(_)
        operation_detail['step_parameter']=step_parameter
        step_triple.append((step_input,operation_detail,step_output))

In [21]:
for item in code_generation[0]['instructions']:
    print(item['text'])

Preheat oven to 350 degrees Fahrenheit.
Spray pan with non stick cooking spray.
Heat milk, water and butter to boiling; stir in contents of both pouches of potatoes; let stand one minute.
Stir in corn.
Spoon half the potato mixture in pan.
Sprinkle half each of cheese and onions; top with remaining potatoes.
Sprinkle with remaining cheese and onions.
Bake 10 to 15 minutes until cheese is melted.
Enjoy !


In [22]:
step_triple

[([],
  {'orginal_step_operation': 'preheat',
   'step_operation': 'Preheat',
   'step_parameter': [('tool', 'oven'), ('temp', '350 degrees F')]},
  'h1'),
 (['pan'],
  {'orginal_step_operation': 'spray',
   'step_operation': 'Spray',
   'step_parameter': [('how', 'non_stick_cooking_spray')]},
  'h2'),
 (['milk', 'water', 'butter'],
  {'orginal_step_operation': 'heat',
   'step_operation': 'Heat',
   'step_parameter': [('how', 'to_boiling')]},
  'h3'),
 (['h3', 'pouches'],
  {'orginal_step_operation': 'stir',
   'step_operation': 'Stir',
   'step_parameter': [('how', 'contents_of_both_pouches_of_potatoes')]},
  'h4'),
 (['h4'],
  {'orginal_step_operation': 'stand',
   'step_operation': 'Make',
   'step_parameter': [('time', 'one_minute')]},
  'h5'),
 (['h5', 'corn'],
  {'orginal_step_operation': 'stir',
   'step_operation': 'Stir',
   'step_parameter': []},
  'h6'),
 (['h6'],
  {'orginal_step_operation': 'spoon',
   'step_operation': 'Place',
   'step_parameter': [('container', 'pan'),

In [23]:
original_input = []
for step in step_triple:
    for ingredient in step[0]:
        if not ingredient.replace('h',"").isdigit():
            original_input.append(ingredient)

In [24]:
operation_list = []
for step in step_triple:
    func_dict = step[1]
    operation_list.append(func_dict['step_operation'])

# Untils

In [19]:
def filterParameter(step_parameter):
    filter_parameter = []
    ### special for ingredient
    ingredient = None
    for item in step_parameter:
        parameter_name = item.split('=')[0]
        parameter_value = item.split('=')[1]
        if parameter_name in Parameter_list:
            filter_parameter.append((parameter_name,parameter_value))
        if parameter_name == 'until':
            filter_parameter.append(('how','until_'+parameter_value))
        if parameter_name == 'heat':
            filter_parameter.append(('temp',parameter_value+'_heat'))
        if parameter_name == 'ingredient':
            ingredient = parameter_value
    return filter_parameter,ingredient